# Analytical Calculation

In [ ]:
import math
import importlib
import os
import random
import pickle
import matplotlib.pyplot as plt
import numpy as np
import itertools
from scipy.stats import pearsonr, spearmanr
from statsmodels.stats import multitest
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
from matplotlib.lines import Line2D
from matplotlib.colors import ListedColormap
from matplotlib.ticker import MultipleLocator, FuncFormatter
from collections import defaultdict
from scipy.stats import linregress
import torch
from AllFnc import eegvislib

import warnings
warnings.filterwarnings("ignore", category = FutureWarning)
warnings.filterwarnings("ignore", category = RuntimeWarning)

# SET PATH
sep         = os.path.sep
dataPath    = '/data/delpup/datasets/eegpickle/'
osPath      = os.path.abspath(os.getcwd())
imgPath     = osPath + sep + 'imgs' + sep
modelsPath  = osPath + sep + 'AlzClassification' + sep + 'Models' + sep
resultsPath = osPath + sep + 'AlzClassification' + sep + 'Results' + sep
numericPath = osPath + sep + 'AnalyticalCalculation' + sep

# IMGS OPTIONS
imgs_format = '.pdf'
save_img    = True

outerFolds  = 10
innerFolds  = 5
classlabels = ['CTL', 'FTD', 'AD']

## Analysis First Layer Initialization

In [ ]:
def weight_change_load(path, split, mod, model, metric):
    """
    Load weight change data and a specific performance metric from pickle files.

    Parameters:
    -----------
    - path (str): Directory path for the model files.
    - split (tuple): Tuple specifying outer and inner folds as (outerFold, innerFold).
    - mod (str): Modifier string to identify the file.
    - model (str): Model identifier.
    - metric (str): Performance metric to load.

    Returns:
    -----------
    - dw (numpy.ndarray): Flattened weight change array scaled by 100.
    - bacc (float): Performance metric value scaled by 100.
    """
    outerFold, innerFold = split

    # Load and process weight change data
    with open(f'{path}{model}_dw_{outerFold}_{innerFold}_{mod}.pkl', 'rb') as file:
        dw_preinit = pickle.load(file)
    dw = np.ravel(dw_preinit.numpy()) * 100  # Flatten and scale

    # Load and process performance metric data
    with open(f'{path}{model}_scores_{outerFold}_{innerFold}_{mod}.pkl', 'rb') as file:
        bacc = pickle.load(file)[metric]
    bacc *= 100  # Scale the performance metric

    return dw, bacc

def weight_change_plot(path, mod, model, splits, spec_dict):
    """
    Plot the histogram of weight changes across specified splits.

    Parameters:
    -----------
    - path (str): Folder to path used by the function weight_change_load.
    - mod (str): Modifier string to identify the file.
    - model (str): Model identifier.
    - splits (list): List of split tuples [(outerFold1, innerFold1), (outerFold2, innerFold2), ...].
    - spec_dict (dict): A dictionary containing various specification parameters 
    and other plotting configurations.

    Returns:
    --------
    - fig (matplotlib.figure.Figure): The generated plot figure.
    """
    fig, ax = plt.subplots(figsize=(spec_dict['figdim'][0], spec_dict['figdim'][1]))
    ax.set_title(spec_dict['title'], fontsize=spec_dict['font'])

    for split in splits:
        # Load metrics for each split
        dw_random, bacc_random = weight_change_load(path, split, mod, model, spec_dict['metric'])
        
        # Clip weight change values to specified limits
        dw_random = np.clip(dw_random, -spec_dict['clip'], spec_dict['clip'])

        # Compute histogram counts and relative frequencies
        counts, bin_edges = np.histogram(dw_random, bins=spec_dict['bins'])
        relative_counts = counts * 100 / len(dw_random)  # Convert to percentage

        # Plot histogram as a bar chart with relative counts
        ax.bar(bin_edges[:-1], relative_counts, width=np.diff(bin_edges), alpha=spec_dict['alpha'],
               label=f'Split {split[0]}-{split[1]}', align='edge')

    # Set axis labels and customize ticks
    ax.set_ylabel('Relative Count %', fontsize=spec_dict['font'] - 2)
    ax.set_xlabel(r'$\Delta_{w\%} = (w - w_{init})/|w|$', fontsize=spec_dict['font'] - 2)
    ax.set_xlim(-spec_dict['clip'], spec_dict['clip'])
    ax.set_xticks(np.arange(-spec_dict['clip'], spec_dict['clip'] + spec_dict['ticks'][0], spec_dict['ticks'][0]))
    ax.set_yticks(np.arange(0, np.max(relative_counts) + spec_dict['ticks'][1], spec_dict['ticks'][1]))
    ax.tick_params(axis='both', labelsize=spec_dict['font'] - 4)
    ax.legend(loc=spec_dict['loc'], fontsize=spec_dict['font'] - 6)

    return fig

In [ ]:
# Example parameters
mod = 'random'
model = 'shn0'
splits = [[9, 2], [6, 5]]
spec_dict = {
    'font': 16,
    'clip': 50,
    'bins': 40,
    'alpha': 0.5,
    'ticks': [10, 2],
    'title': 'Weights Change in the First Layer ShallowNet',
    'figdim': [7, 5],
    'loc': 'upper right',
    'metric': 'accuracy_weighted',
    'filename': 'weightChangeFirstLayer'
}

# Generate and optionally save the plot
fig = weight_change_plot(numericPath + 'Initialization' + sep, mod, model, splits, spec_dict)

if save_img:
    fig.savefig(imgPath + spec_dict['filename'] + imgs_format,
                transparent=False, bbox_inches='tight')

## Analysis Topographies Correlation

In [ ]:
def pvals_calculation(scalps, indexs, spec_dict):
    """
    Calculate the pairwise p-values and correlation statistics between filter activations across specified scalp maps.
    Adjusts p-values for multiple comparisons and returns the corrected p-value matrix and a significance matrix.

    Parameters:
    ----------
    - scalps (list of np.ndarray): List of 4D arrays representing the scalp maps. 
    Each array should have a shape (filters, kernels=1, hight=channels, width=1)    
    - indexs (list of int): Indices for two specific scalp maps in `scalps` to compare.
    - spec_dict (dict): A dictionary containing various specification parameters 
    and other plotting configurations.
    
    Returns:
    -------
    - corrected_pvals (np.ndarray): Symmetric matrix of corrected p-values for the pairwise filter comparisons.
    - significance_matrix (np.ndarray): Boolean matrix indicating where p-values are significant after correction.
    - stats (np.ndarray): Symmetric matrix of pearson coefficients for the pairwise filter comparisons.
    """
    
    filters = scalps[0].shape[0]  # Number of filters in each scalp map
    pvals = np.zeros((filters, filters))  # Matrix to store p-values
    stats = np.zeros((filters, filters))  # Matrix to store correlation statistics

    # Calculate pairwise Pearson correlations between filters in specified scalp maps
    for i in range(filters):
        for j in range(filters):
            res = pearsonr(scalps[indexs[0]][i, 0, :, 0], scalps[indexs[1]][j, 0, :, 0])
            pvals[i, j] = res.pvalue
            stats[i, j] = res.statistic

    # Extract the upper triangle of p-values for multiple testing correction
    upper_tri_indices = np.triu_indices(filters)
    flattened_pvals = pvals[upper_tri_indices].flatten()

    # Apply multiple testing correction (e.g., Bonferroni, FDR)
    corrected_res = multitest.multipletests(flattened_pvals, alpha=spec_dict['pval'], method=spec_dict['method'])
    corrected_pvals_flat = corrected_res[1]  # Corrected p-values
    significance_matrix_flat = corrected_res[0]  # Boolean array for significance

    # Reconstruct matrices with corrected p-values and significance flags in original dimensions
    corrected_pvals = np.zeros((filters, filters))
    significance_matrix = np.zeros((filters, filters), dtype=bool)

    # Map corrected values to the upper triangle
    corrected_pvals[upper_tri_indices] = corrected_pvals_flat
    significance_matrix[upper_tri_indices] = significance_matrix_flat

    # Reflect upper triangle to lower triangle for symmetry
    corrected_pvals = corrected_pvals + corrected_pvals.T - np.diag(np.diag(corrected_pvals))
    significance_matrix = significance_matrix | significance_matrix.T

    return corrected_pvals, significance_matrix, stats


def pvals_plot(corrected_pvals, significance_matrix, stats, splits, indexs, spec_dict):
    """
    Plot the significance matrix with annotated corrected p-values for significant comparisons.

    Parameters:
    ----------
    - corrected_pvals (np.ndarray): Symmetric matrix of corrected p-values for filter comparisons.
    - significance_matrix (np.ndarray): Boolean matrix indicating where p-values are significant.
    - stats (np.ndarray): Symmetric matrix of pearson coefficients for the pairwise filter comparisons.
    - splits (list of lists): List of split identifiers. 
    Each identifier is a list of integers representing the split structure.
    - indexs (list of int): Indices for two specific scalp maps from `splits` to label in the plot.
    - spec_dict (dict): A dictionary containing various specification parameters 
    and other plotting configurations.

    Returns:
    -------
    fig (matplotlib.figure.Figure): The resulting figure with the significance matrix plot.
    """
    
    filters = np.shape(corrected_pvals)[0]  # Number of filters

    # Initialize plot
    fig, ax = plt.subplots(figsize=(spec_dict['figdim'][0], spec_dict['figdim'][1]))
    ax.imshow(significance_matrix, cmap='Greys')  # Display significance matrix
    ax.set_ylabel(f'Split {splits[indexs[0]][0]}-{splits[indexs[0]][1]}', fontsize=spec_dict['font']-2)
    ax.set_xlabel(f'Split {splits[indexs[1]][0]}-{splits[indexs[1]][1]}', fontsize=spec_dict['font']-2)
    ax.set_xticks(np.arange(0, filters, 1))
    ax.set_xticklabels(spec_dict['xticks'], fontsize=spec_dict['font']-4)
    ax.set_yticks(np.arange(0, filters, 1))
    ax.set_yticklabels(spec_dict['xticks'], fontsize=spec_dict['font']-4)
    ax.set_title(spec_dict['title'], fontsize=spec_dict['font'])

    # Annotate significant p-values on the plot
    for i in range(filters):
        for j in range(filters):
            if significance_matrix[i, j]:  # Only annotate cells with significant p-values
                adj_R2 = eegvislib.adjusted_R2(stats[i,j], spec_dict['C'], 1)
                ax.text(j, i, f'p$={corrected_pvals[i, j]:.1e}$ \n $\\rho({spec_dict['C']})={stats[i,j]:.2f}$ \n adj.$R^2={adj_R2:.2f}$', 
                        ha='center', va='center', color='white', fontsize=spec_dict['font']-8)

    return fig

In [ ]:
splits = [[9, 2], [6, 5]]
modelsToimport = ['alz_flt_125_shn7db_009_002_000050_019_004','alz_flt_125_shn7db_006_005_000050_019_004']

spec_dict = {'pval': 0.05,
             'method': 'holm',
             'xticks': ['$\\delta$', '$\\theta$', '$\\alpha$', '$\\beta_1$',
                       '$\\beta_2$', '$\\beta_3$', '$\\gamma$'],
             'C': 19,
             'title': f'Significance Correlation (Holm corrected)',
             'font': 16,
             'figdim': [7, 7]}

scalps = []
for modelToimport in modelsToimport:
    shnm = torch.load(modelsPath + modelToimport + '.pt')    
    scalps.append(shnm['encoder.conv2.weight'].numpy())

corrected_pvals, significance_matrix, stats = pvals_calculation(scalps, [0,1], spec_dict)
fig = pvals_plot(corrected_pvals, significance_matrix, stats, splits, [0,1], spec_dict)

In [ ]:
corrected_pvals, significance_matrix, stats = pvals_calculation(scalps, [0,0], spec_dict)
fig = pvals_plot(corrected_pvals, significance_matrix, stats, splits, [0,0], spec_dict)

In [ ]:
corrected_pvals, significance_matrix, stats = pvals_calculation(scalps, [1,1], spec_dict)
fig = pvals_plot(corrected_pvals, significance_matrix, stats, splits, [1,1], spec_dict)

## Analysis Overlap Embedding

In [ ]:
model = 'shn7db' #'shn7db'
shns = []
acc = []
for i in range(1,outerFolds+1):
    for j in range(1,innerFolds+1):
        file_suffix = f'00{i}_00{j}' if i != 10 else f'0{i}_00{j}'
        filename = f'alz_flt_125_{model}_{file_suffix}_000050_019_004'
        with open(numericPath + 'Overlap' +sep + 'Results' +sep + filename + '.pickle', 'rb') as f:
            shn = pickle.load(f)
        shns.append(shn)
        with open(resultsPath + filename + '.pickle', 'rb') as f:
            shnR = pickle.load(f)
        acc.append(shnR['accuracy_weighted'])

key_list = list(shn.keys())

In [ ]:
overlap_matrix = np.zeros((len(shns),len(key_list)))
for j in range(len(key_list)):
    for i in range(len(shns)):
        overlap_matrix[i,j] = shns[i][key_list[j]]

# Add dependent variable safely
keys = [f"{tup[0]}_{tup[1]}" for tup in key_list]
df = pd.DataFrame(overlap_matrix.copy(),columns = keys)
df["acc"] = pd.Series(acc, index=df.index)  # Ensures alignment

def transform_label(label):
    parts = label.split('_')
    return f"Separability: ${parts[0]}_{parts[1]}~-~{parts[2]}_{parts[3]}$"

In [ ]:
print(df['C_t_C_T'].mean(),df['C_t_C_T'].std())
print(df['F_t_F_T'].mean(),df['F_t_F_T'].std())
print(df['A_t_A_T'].mean(),df['A_t_A_T'].std())

print(df['A_t_C_t'].mean(),df['A_t_C_t'].std())
print(df['F_t_C_t'].mean(),df['F_t_C_t'].std())
print(df['A_t_F_t'].mean(),df['A_t_F_t'].std())

In [ ]:
# Correct formula syntax
model = smf.ols("acc ~ " + " + ".join(keys), data=df)
results = model.fit()

print("\nRegression Results:")
print(results.summary())

# Prune based on p-value
pruned_features = [feature for feature, p_value in zip(keys, results.pvalues[1:]) if p_value < 0.05]
print(f"Pruned features: {pruned_features}")

dfc = df[keys].copy()
# Add constant (intercept) to the data
dfc = sm.add_constant(dfc)
vif_data = pd.DataFrame()

# Calculate VIF for each feature (excluding the constant column)
vif_data["feature"] = dfc.columns[1:]  # Excluding the constant column
vif_data["VIF"] = [round(variance_inflation_factor(dfc.values, i), 4) for i in range(1, dfc.shape[1])]

# Sort VIF values in descending order
vif_data = vif_data.sort_values(by="VIF", ascending=False)
vif_data

In [ ]:
# Add dependent variable safely
keys = [col for col in df.columns[:-1] if 'v' not in col]
dfc  = df[keys]

# Correct formula syntax
model = smf.ols("acc ~ " + " + ".join(keys), data=dfc)
results = model.fit()

print("\nRegression Results:")
print(results.summary())

# Prune based on p-value
pruned_features = [feature for feature, p_value in zip(keys, results.pvalues[1:]) if p_value < 0.05]
print(f"Pruned features: {pruned_features}")

# Add constant (intercept) to the data
dfc = sm.add_constant(dfc)
vif_data = pd.DataFrame()

# Calculate VIF for each feature (excluding the constant column)
vif_data["feature"] = dfc.columns[1:]  # Excluding the constant column
vif_data["VIF"] = [round(variance_inflation_factor(dfc.values, i), 4) for i in range(1, dfc.shape[1])]

# Sort VIF values in descending order
vif_data = vif_data.sort_values(by="VIF", ascending=False)
vif_data

In [ ]:
# Add dependent variable safely
keys = [col for col in df.columns[:-1] if 'T' not in col]

dfc = df[keys]

# Correct formula syntax
model = smf.ols("acc ~ " + " + ".join(keys), data=dfc)
results = model.fit()

print("\nRegression Results:")
print(results.summary())

# Prune based on p-value
pruned_features = [feature for feature, p_value in zip(keys, results.pvalues[1:]) if p_value < 0.05]
print(f"Pruned features: {pruned_features}")

# Add constant (intercept) to the data
dfc = sm.add_constant(dfc)
vif_data = pd.DataFrame()

# Calculate VIF for each feature (excluding the constant column)
vif_data["feature"] = dfc.columns[1:]  # Excluding the constant column
vif_data["VIF"] = [round(variance_inflation_factor(dfc.values, i), 4) for i in range(1, dfc.shape[1])]

# Sort VIF values in descending order
vif_data = vif_data.sort_values(by="VIF", ascending=False)
vif_data

In [ ]:
# Add dependent variable safely
keys = [col for col in df.columns[:-1] if 't' not in col]

dfc = df[keys]

# Correct formula syntax
model = smf.ols("acc ~ " + " + ".join(keys), data=dfc)
results = model.fit()

print("\nRegression Results:")
print(results.summary())

# Prune based on p-value
pruned_features = [feature for feature, p_value in zip(keys, results.pvalues[1:]) if p_value < 0.05]
print(f"Pruned features: {pruned_features}")

# Add constant (intercept) to the data
dfc = sm.add_constant(dfc)
vif_data = pd.DataFrame()

# Calculate VIF for each feature (excluding the constant column)
vif_data["feature"] = dfc.columns[1:]  # Excluding the constant column
vif_data["VIF"] = [round(variance_inflation_factor(dfc.values, i), 4) for i in range(1, dfc.shape[1])]

# Sort VIF values in descending order
vif_data = vif_data.sort_values(by="VIF", ascending=False)
vif_data

### Correlation + Forward-Backward Selection

In [ ]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out=0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    """
    included = list(initial_list)
    
    while True:
        changed = False
        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=float)

        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]

        if not new_pval.empty:
            best_feature = new_pval.idxmin()  # Get column name instead of index
            best_pval = new_pval.min()
            if best_pval < threshold_in:
                included.append(best_feature)
                changed = True
                if verbose:
                    print(f'Add  {best_feature:30} with p-value {best_pval:.6f}')

        # Backward step
        if included:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
            pvalues = model.pvalues.iloc[1:]  # Exclude intercept
            
            if not pvalues.empty:
                worst_feature = pvalues.idxmax()  # Get column name instead of index
                worst_pval = pvalues.max()

                if worst_pval > threshold_out:
                    included.remove(worst_feature)
                    changed = True
                    if verbose:
                        print(f'Drop {worst_feature:30} with p-value {worst_pval:.6f}')
        
        if not changed:
            break

    return included
    
def correlation_pvalues(df, target_col='acc', method='fdr_bh'):
    # Extract all columns and initialize the p-value matrix
    cols = df.columns
    pval_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)
    pvals = []

    # Compute p-values for correlation between 'acc' and all other columns
    for col in cols:
        if col != target_col:  # Skip the 'acc' column itself
            corr, pval = pearsonr(df[col],df[target_col])
            pvals.append(pval)
            pval_matrix.loc[target_col, col] = pval
            pval_matrix.loc[col, target_col] = pval  # Mirror the matrix

    # Apply multiple comparison correction to the p-values
    adjusted_pvals = multitest.multipletests(pvals, method=method)[1]  # Corrected p-values

    # Update the p-value matrix with the adjusted p-values
    index = 0
    for col in cols:
        if col != target_col:  # Skip the 'acc' column itself
            pval_matrix.loc[target_col, col] = adjusted_pvals[index]
            pval_matrix.loc[col, target_col] = adjusted_pvals[index]
            index += 1

    return pval_matrix    

In [ ]:
f = 'C_t_C_T'
print(pearsonr(df[f], df['acc']))
plt.scatter(df['acc'],df[f])
plt.show()

In [ ]:
# Example usage
p_values_corrected = correlation_pvalues(df,target_col='acc', method='holm')  # 'bonferroni' for stricter correction
significant_features = p_values_corrected['acc'][p_values_corrected['acc'] < 0.05].index.tolist()

# Example Usage
X = df.drop(columns=['acc'])  # Features
y = df['acc']                 # Target variable
resultF = stepwise_selection(X[significant_features], y, significant_features)
print(resultF)

In [ ]:
print(df.corr().loc['acc',:].loc[significant_features])
print(p_values_corrected['acc'][p_values_corrected['acc'] < 0.05])

In [ ]:
keys = resultF
dfc = df[keys].copy()

# Correct formula syntax
model = smf.ols("acc ~ " + " + ".join(keys), data=dfc)
results = model.fit()

print("\nRegression Results:")
print(results.summary())

# Add constant (intercept) to the data
dfc = sm.add_constant(dfc)

vif_data = pd.DataFrame()

# Calculate VIF for each feature (excluding the constant column)
vif_data["feature"] = dfc.columns[1:]  # Excluding the constant column
vif_data["VIF"] = [round(variance_inflation_factor(dfc.values, i), 4) for i in range(1, dfc.shape[1])]

# Sort VIF values in descending order
vif_data = vif_data.sort_values(by="VIF", ascending=False)
vif_data

### Variables Visualization

In [ ]:
# Create a 2x2 grid of subplots
spec_dict = {'font':12,
             'linew':1,
             'figdim': [12,10],
             'figname': 'shn0_overlap'}

fig, axes = plt.subplots(2, 2, figsize=(spec_dict['figdim'][0], spec_dict['figdim'][1]))

# Flatten the axes array for easier indexing
axes = axes.flatten()

# Loop over pairwise combinations and plot in the subplots
index = 0
for i in range(len(keys)):
    for j in range(i + 1, len(keys)):
        # Scatter plot for each pairwise combination
        ax = axes[index]
        scatter = ax.scatter(df[keys[i]], df[keys[j]], c=[i*100 for i in df['acc']], cmap='jet')
        
        # Add vertical and horizontal lines at the value 1
        ax.axvline(1, linestyle='--', color='k', linewidth=1)
        ax.axhline(1, linestyle='--', color='k', linewidth=1)
        
        # Set labels for the axes
        ax.set_xlabel(transform_label(keys[i]),fontsize=spec_dict['font']-2)
        ax.set_ylabel(transform_label(keys[j]),fontsize=spec_dict['font']-2)
        ax.set_title('ShallowNet',fontsize=spec_dict['font'])
        
        # Add colorbar to the scatter plot
        fig.colorbar(scatter, ax=ax, label='Weighted Accuracy %')

        # Increment the index for the next subplot
        index += 1

# Remove the last empty subplot (since we have 3 plots for 3 variables)
for i in range(index,2*2):
    fig.delaxes(axes[i])

if save_img:
    fig.savefig(imgPath + spec_dict['figname'] + imgs_format,
                transparent=False, bbox_inches='tight')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
keys = resultF.copy()
keys.append('acc')
# Create a 2x2 grid of subplots
spec_dict = {'font':12,
             'linew':1,
             'figdim': [12,10],
             'figname': 'shn7db_overlap'}

fig, axes = plt.subplots(2, 2, figsize=(spec_dict['figdim'][0], spec_dict['figdim'][1]))

# Flatten the axes array for easier indexing
axes = axes.flatten()

# Loop over pairwise combinations and plot in the subplots
index = 0
for i in range(len(keys)):
    for j in range(i + 1, len(keys)):
        # Scatter plot for each pairwise combination
        ax = axes[index]
        scatter = ax.scatter(df[keys[i]], df[keys[j]], c=[i*100 for i in df['acc']], cmap='jet')
        
        # Add vertical and horizontal lines at the value 1
        ax.axvline(1, linestyle='--', color='k', linewidth=1)
        #ax.axhline(1, linestyle='--', color='k', linewidth=1)
        
        # Set labels for the axes
        ax.set_xlabel(transform_label(keys[i]),fontsize=spec_dict['font']-2)
        ax.set_ylabel('Weighted Accuracy',fontsize=spec_dict['font']-2)
        ax.set_title('xEEGNet',fontsize=spec_dict['font'])
        
        # Add colorbar to the scatter plot
        fig.colorbar(scatter, ax=ax, label='Weighted Accuracy %')

        # Increment the index for the next subplot
        index += 1

# Remove the last empty subplot (since we have 3 plots for 3 variables)
for i in range(index,2*2):
    fig.delaxes(axes[i])

if save_img:
    fig.savefig(imgPath + spec_dict['figname'] + imgs_format,
                transparent=False, bbox_inches='tight')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

## Analysis Model Variation

In [ ]:
models = ["shn0", "shn1", "shn2", "shn3", "shn4", "shn5", "shn67",
          "shn628","shn663","shn6119","shn6126","shn6127","shn7","shn7db"]

# Set of models for which a specific condition applies
model_set = {'shn0', 'shn1', 'shn2', 'shn3'} #These models has not the first layer frozen

spec_dict = {'figname': 'modelVarAccvsWeight',
             'font': 18,
             'linew': 2,
             's':4,
             'rotation': 45,
             'loc': 'upper center',
             'title': 'Architecture Variation - Performance',
             'accrandom': 1/len(classlabels),
             'linestyle': '--',
             'markers': ['X','o'],
             'ylima': [20,105],
             'ylime': [0,200],
             'ylimw': [10**1,10**6],
             'jitter': 0.02,
             'figdim': [15,5],
             'color': ['tab:orange','tab:blue']}

In [ ]:
# Loop over each model
total_weights_v = []
for model in models:
    filename = f'alz_flt_125_{model}_001_001_000050_019_004'
    shn = torch.load(modelsPath + filename + '.pt')

    total_weights = 0
    # Iterate through the state_dict (model weights)
    for key, tensor in shn.items():
        # Skip specific keys related to batch normalization
        if key in ['encoder.batch1.running_mean', 'encoder.batch1.running_var', 'encoder.batch1.num_batches_tracked']:
            continue
        
        # Add the tensor's total number of elements, applying conditions for specific models
        if model in model_set:
            total_weights += tensor.numel()
        elif key not in ['encoder.conv1.weight', 'encoder.conv1.bias']:
            total_weights += tensor.numel()
    
    total_weights_v.append(total_weights)

sorted_pairs = sorted(zip(total_weights_v, models))
stored = sorted_pairs[0]
sorted_pairs[0] = sorted_pairs[1]
sorted_pairs[1] = stored

# Unzip the sorted pairs to get sorted vector1
models = [v for _, v in sorted_pairs]
total_weights_v = [v for v, _ in sorted_pairs]

print(total_weights_v)

models1 = [sorted_pairs[i][1] for i in range(len(models))]
# models1[9] = 'ShallowNet'
# models1[0] = 'Med-ShallowNet'

models1 = ['xEEGNet',
 'shn7',
 'shn$6_{7}$',
 'shn$6_{28}$',
 'shn$6_{63}$',
 'shn$6_{119}$',
 'shn$6_{126}$',
 'shn$6_{127}$',
 'shn5',
 'ShallowNet',
 'shn4',
 'shn1',
 'shn2',
 'shn3']

In [ ]:
acc_dict = defaultdict(list)
acc_train = {}
acc_val = {}
# Precompute accuracy values for each model (only once)
for model in models:
    acct = []  # Reset accuracy list for each model
    accv = []
    for i in range(1, outerFolds+1):  # Loop over first parameter
        for j in range(1, innerFolds+1):  # Loop over second parameter
            file_suffix = f'00{i}_00{j}' if i != 10 else f'0{i}_00{j}'
            filename = f'alz_flt_125_{model}_{file_suffix}_000050_019_004'

            # Load accuracy data from pickle file (avoid reloading for each iteration)
            with open(resultsPath + filename + '.pickle', 'rb') as f:
                shn = pickle.load(f)

            # Append the accuracy to the list
            acc_dict[model].append(shn['accuracy_weighted'] * 100)
            acct.append(shn['training_loss_curve'])
            accv.append(shn['validation_loss_curve'])

    # Append the accuracy to the list
    acc_train[model] = acct
    acc_val[model] = accv
    print(f'{model} QCV: {eegvislib.quartile_coefficient_of_variation(acc_dict[model]):.4f}')

### Model Weights vs Accuracy

In [ ]:
fig, ax1 = plt.subplots(1, 1, 
                        figsize=(spec_dict['figdim'][0], spec_dict['figdim'][1]))
ax2 = ax1.twinx()

# Plot the weights vs models (on the second y-axis, ax2)
ax2.plot(range(1, len(models) + 1), total_weights_v, color=spec_dict['color'][0], 
         marker=spec_dict['markers'][0], linestyle=spec_dict['linestyle'], 
         linewidth=spec_dict['linew'], label='Weights')

# Plot boxplots for all models (on the first y-axis, ax1)
box = ax1.boxplot(list(acc_dict.values()), showfliers=False)

# Remove median line from boxplot (using more efficient approach)
for median in box['medians']:
    median.set_visible(False)

# Add scatter points (individual accuracy values) for each model
for i, model in enumerate(models):
    acc = acc_dict[model]
    ax1.scatter(np.ones(len(acc)) * (i + 1) + spec_dict['jitter']*np.random.randn(len(acc)), acc,
                color='k', alpha=1, s=spec_dict['s'])

# Label the first y-axis (for accuracies)
ax1.set_ylabel("Median Weighted Accuracy %", color=spec_dict['color'][1], fontsize=spec_dict['font']-2)
ax1.set_ylim(spec_dict['ylima'][0], spec_dict['ylima'][1])
ax1.set_xlim(0.5, len(models) + 0.75)

# Plot the median value as text on the boxplot for each model
median_v = [np.median(acc_dict[model]) for model in models]
for i, median in enumerate(median_v):
    ax1.text(i + 1, spec_dict['ylima'][0]+2, f'{median:.1f}%', ha='center', va='bottom', 
             color=spec_dict['color'][1], fontsize=spec_dict['font']-6, fontweight='bold')

ax1.plot(np.arange(1, 1 + len(models)), median_v, marker=spec_dict['markers'][1], linestyle=spec_dict['linestyle'],
         color=spec_dict['color'][1], linewidth=spec_dict['linew'])

# Label the second y-axis (for weights)
ax2.set_yscale('log')
ax2.set_ylabel("$N^{\\circ}$ Trainable Parameters", color=spec_dict['color'][0], fontsize=spec_dict['font']-2)
ax2.set_ylim(spec_dict['ylimw'][0], spec_dict['ylimw'][1])

# Add a horizontal line at 33.33 for reference
ax1.axhline(spec_dict['accrandom']*100, linestyle=spec_dict['linestyle'], linewidth=spec_dict['linew'], color='red')

# Add a title to the plot
ax1.set_title(spec_dict['title'], fontsize=spec_dict['font'])

# Create legend
legend_elements = [Line2D([0], [0], marker='.', color='k', label='Single Split', linestyle='None'),
                   Line2D([0], [0], label=f'{spec_dict['accrandom']:.0%} Random Guess', 
                          linestyle=spec_dict['linestyle'], linewidth=spec_dict['linew'], color='red')]
ax1.legend(handles=legend_elements, loc=spec_dict['loc'], fontsize=spec_dict['font']-6)

# Set x-tick labels for models
ax1.set_xticklabels(models1, rotation=spec_dict['rotation'])
ax1.tick_params(axis='both', which='major', labelsize=spec_dict['font']-4)
ax2.tick_params(axis='both', which='major', labelsize=spec_dict['font']-4)

# Label the figure
ax1.text(len(models), spec_dict['ylima'][1]-10, '$(A)$', fontsize=spec_dict['font'])

# Save the image if needed
if save_img:
    fig.savefig(imgPath + spec_dict['figname'] + imgs_format,
                transparent=False, bbox_inches='tight')

In [ ]:
median_v = [np.median(acc_dict[model]) for model in models]
median_v

In [ ]:
np.mean(acc_dict['shn0'])-np.mean(acc_dict['shn7db'])

In [ ]:
np.max(acc_dict['shn7db'])

In [ ]:
# Perform linear regression
slope, intercept, r_value, p_value, std_err = linregress(np.log(total_weights_v), median_v)

adj_R2 = eegvislib.adjusted_R2(r_value, len(median_v), 1)

# Print the results
print(f' r({len(median_v)-2:.0f})={r_value:.2f}, p={p_value:.3f}, Adj. R^2={adj_R2:.2f}')

### Model Weights vs Number of Epochs

In [ ]:
spec_dict['figname'] = 'modelVarEpochsvsWeight'
spec_dict['title']   = "Architecture Variation - Training Length"
spec_dict['color']   = ['tab:orange','tab:green']
spec_dict['ylima']   = [-15, 200]
patience = 15

fig, ax1 = plt.subplots(1, 1, 
                        figsize=(spec_dict['figdim'][0], spec_dict['figdim'][1]))

# Create a secondary y-axis for weights
ax2 = ax1.twinx()

# Plot weights on the secondary y-axis (ax2)
ax2.plot(range(1, len(models) + 1), total_weights_v, color=spec_dict['color'][0], marker=spec_dict['markers'][0], 
         linestyle=spec_dict['linestyle'], linewidth=spec_dict['linew'], label='Weights')

# Precompute n_epoch for each model
n_epoch = {
    model: [len(acc_train[model][i]) - patience + 1 for i in range(int(outerFolds*innerFolds))]
    for model in models}

# Boxplot for n_epoch
box = ax1.boxplot([n_epoch[model] for model in models], labels=models, showfliers=False)

# Remove median lines from the boxplot
for median in box['medians']:
    median.set_visible(False)

# Scatter plot individual epoch values with slight random offsets on the x-axis
for i, model in enumerate(models):
    nepoch = n_epoch[model]
    ax1.scatter(np.ones(len(nepoch)) * (i + 1) + spec_dict['jitter'] * np.random.randn(len(nepoch)), nepoch,
                color='k', alpha=1, s=spec_dict['s'])

# Label the first y-axis (for epochs)
ax1.set_ylabel("Median $~N^{\\circ}$ Epochs", color=spec_dict['color'][1], fontsize=spec_dict['font']-2)
ax1.set_ylim(spec_dict['ylima'][0],spec_dict['ylima'][1])
ax1.set_xlim(0.5, len(models) + 0.75)

# Compute and plot the median values
median_v = [np.median(n_epoch[model]) for model in models]
for i, median in enumerate(median_v):
    ax1.text(i + 1, spec_dict['ylima'][0]+2, f'{median:.0f}', ha='center', va='bottom', 
             color=spec_dict['color'][1], fontsize=spec_dict['font']-6, fontweight='bold')

ax1.plot(np.arange(1, 1 + len(models)), median_v, marker=spec_dict['markers'][1], 
         linestyle=spec_dict['linestyle'], color=spec_dict['color'][1], linewidth=spec_dict['linew'])

# Label the second y-axis (for weights)
ax2.set_yscale('log')
ax2.set_ylabel("$N^{\\circ}$ Trainable Parameters", color=spec_dict['color'][0], fontsize=spec_dict['font']-2)
ax2.set_ylim(spec_dict['ylimw'][0],spec_dict['ylimw'][1])

# Add x-tick labels for models
ax1.set_xticklabels(models1, rotation=spec_dict['rotation'])

# Add title and other plot decorations
ax1.set_title(spec_dict['title'], fontsize=spec_dict['font'])
ax1.tick_params(axis='both', which='major', labelsize=spec_dict['font']-4)
ax2.tick_params(axis='both', which='major', labelsize=spec_dict['font']-4)

# Create the legend
legend_elements = [Line2D([0], [0], marker='.', color='k', label='Single Split', linestyle='None')]
ax1.legend(handles=legend_elements, loc='upper center', fontsize=spec_dict['font']-6)

# Annotate the plot with figure label
ax1.text(len(models), spec_dict['ylime'][1]-20, '$(B)$', fontsize=spec_dict['font'])

if save_img:
    fig.savefig(imgPath + spec_dict['figname'] + imgs_format,
                transparent=False, bbox_inches='tight')

In [ ]:
# Perform linear regression
slope, intercept, r_value, p_value, std_err = linregress(np.log(total_weights_v), median_v)

adj_R2 = eegvislib.adjusted_R2(r_value, len(median_v), 1)

# Print the results
print(f' r({len(median_v)-2:.0f})={r_value:.2f}, p={p_value:.3f}, Adj. R^2={adj_R2:.2f}')

## Analysis Loss Curves

In [ ]:
models      = ['shn0', 'shn5', 'shn6127', 'shn628', 'shn7db']

spec_dict = {}
spec_dict['models']     = ['ShallowNet', 'shn5', 'shn6$_{127}$', 'shn6$_{28}$', 'xEEGNet']
spec_dict['listcolors'] = ['black','tab:brown', 'tab:red', 'tab:orange', 'tab:green']
spec_dict['font']     = 16
spec_dict['figname']  = 'lossCurves'
spec_dict['linew']    = 0.5
spec_dict['figdim']   = [10,6]
spec_dict['titlet']   = 'Train Loss Curves'
spec_dict['titlev']   = 'Validation Loss Curves'
spec_dict['titlec']   = 'Correlation Train-Val Loss'
spec_dict['loc']      = 'upper right'
spec_dict['xlimt']    = [0.9, 300]
spec_dict['ylimt']    = [0.0, 3.0]
spec_dict['xlimv']    = [0.9, 300]
spec_dict['ylimv']    = [0.5, None]
spec_dict['xlimc']    = [-1,1]
spec_dict['ylimc']    = [0,50]
spec_dict['ytick']    = 5
spec_dict['xtick']    = 0.2
spec_dict['bin']      = 0.1
spec_dict['rotation'] = 45

acc_train = {}
acc_val = {}
# Precompute accuracy values for each model (only once)
for model in models:
    acct = []  # Reset accuracy list for each model
    accv = []
    for i in range(1, outerFolds+1):  # Loop over first parameter
        for j in range(1, innerFolds+1):  # Loop over second parameter
            file_suffix = f'00{i}_00{j}' if i != 10 else f'0{i}_00{j}'
            filename = f'alz_flt_125_{model}_{file_suffix}_000050_019_004'

            # Load accuracy data from pickle file (avoid reloading for each iteration)
            with open(resultsPath + filename + '.pickle', 'rb') as f:
                shn = pickle.load(f)

            # Append the accuracy to the list
            acct.append(shn['training_loss_curve'])
            accv.append(shn['validation_loss_curve'])

    # Append the accuracy to the list
    acc_train[model] = acct
    acc_val[model] = accv

fig = eegvislib.overfitting_inspection(models, acc_train, acc_val, spec_dict)

if save_img:
    fig.savefig(imgPath + spec_dict['figname'] + imgs_format,
                transparent=False, bbox_inches='tight')

In [ ]:
importlib.reload(eegvislib)

## Analysis Dense Layer

In [ ]:
model = 'shn7db'
f_lim = [1,45]
spec_dict = {'font':12,
             'linew':1,
             'cmap':'RdBu_r',
             'bands': {'$\\delta$':  [0,4],
                       '$\\theta$':  [4,8],
                       '$\\alpha$':  [8,12],
                       '$\\beta_1$': [12,16],
                       '$\\beta_2$': [16,20],
                       '$\\beta_3$': [20,28],
                       '$\\gamma$': [28,f_lim[1]]},
             'classlabels': classlabels,
             'figdim': [4,1],
             'plot_type': 'second',
             'cmap': 'RdBu_r',
             'figname': 'uniqueConfigDense'}

spec_dict['xticks'] = [i for i in spec_dict['bands'].keys()]

In [ ]:
matrix_weights = []
for i in range(1,outerFolds+1):
    for j in range(1,innerFolds+1):
        file_suffix = f'00{i}_00{j}' if i != 10 else f'0{i}_00{j}'
        filename = f'alz_flt_125_{model}_{file_suffix}_000050_019_004'
        shn = torch.load(modelsPath + filename + '.pt')
        fig, mask = eegvislib.denseweights_plot(shn,'Dense',spec_dict)
        matrix_weights.append(mask)
        plt.close()
        
matrix_count = {}
# Count occurrences of each matrix
for matrix in matrix_weights:
    matrix_tuple = tuple(map(tuple, matrix))  # Convert to a hashable format
    if matrix_tuple in matrix_count:
        matrix_count[matrix_tuple] += 1
    else:
        matrix_count[matrix_tuple] = 1

# Convert counts back to a list of matrices
unique_matrices = [np.array(matrix) for matrix in matrix_count.keys()]
counts = list(matrix_count.values())

cmap = ListedColormap(['white', 'red', 'blue', 'green'])

# Now you have a list of unique matrices
fig, ax = plt.subplots(len(unique_matrices),1,
                       figsize=(spec_dict['figdim'][0], spec_dict['figdim'][1]*len(unique_matrices)),
                       constrained_layout=True)
for i in range(len(unique_matrices)):
    ax[i].set_title(f'Number of Splits with this config.: {counts[i]}', fontsize=spec_dict['font'])
    ax[i].imshow(unique_matrices[i], aspect='auto', 
                 cmap=cmap, interpolation='nearest')
    ax[i].set_yticks(np.arange(0, 3))
    ax[i].set_yticklabels(labels=spec_dict['classlabels'], fontsize=spec_dict['font']-4)
    ax[i].set_xticks(np.arange(0, len(spec_dict['xticks']), 1))
    ax[i].set_xticklabels(spec_dict['xticks'], fontsize=spec_dict['font']-4)

if save_img:
    fig.savefig(imgPath + spec_dict['figname'] + imgs_format,
                transparent=False, bbox_inches='tight')

## Analysis Statical Models Comparison

In [ ]:
# Function to load accuracy data for each model
def load_accuracy_data(model, path, folds, task='alz', pipe='flt', srate='125', lr='000050', ch='019', w='004', metric='accuracy_weighted'):
    acc = []
    for i in range(folds[0][0], folds[0][1]+1):  # Loop over first parameter
        for j in range(folds[1][0], folds[1][1]+1):  # Loop over second parameter
            file_suffix = f'00{i}_00{j}' if i != 10 else f'0{i}_00{j}'
            filename = f'{task}_{pipe}_{srate}_{model}_{file_suffix}_{lr}_{ch}_{w}'
            with open(path + filename + '.pickle', 'rb') as f:
                shn = pickle.load(f)
            if isinstance(shn, dict):
                acc.append(shn['accuracy_weighted'])
            else:
                acc.append(shn)
    return acc

In [ ]:
metrics = {}
folds = [[1,outerFolds],[1,innerFolds]]
task='alz'
pipe='flt'
srate='125'
model = 'shn7db'
lr='000050'
ch='019'
w='004'
accMLRM = []
accshnMLRM = []
path = numericPath + 'StatisticalModels' + sep + 'Results' + sep
for i in range(folds[0][0], folds[0][1]+1):  # Loop over first parameter
    for j in range(folds[1][0], folds[1][1]+1):  # Loop over second parameter
        file_suffix = f'00{i}_00{j}' if i != 10 else f'0{i}_00{j}'
        filename = f'{task}_{pipe}_{srate}_{model}_{file_suffix}_{lr}_{ch}_{w}'
        with open(path + filename + '.pickle', 'rb') as f:
            shn = pickle.load(f)
        accMLRM.append(shn['MLRM_accuracy_weighted'])
        accshnMLRM.append(shn['shnMLRM_accuracy_weighted'])

metrics['MLRM'] = accMLRM
metrics['shnMLRM'] = accshnMLRM

model = 'shn7db'
path = resultsPath
metrics['shn7db'] = load_accuracy_data(model, resultsPath, [[1,outerFolds],[1,innerFolds]])

model = 'shn0'
path = resultsPath
metrics['shn0'] = load_accuracy_data(model, resultsPath, [[1,outerFolds],[1,innerFolds]])

In [ ]:
spec_dict = {'figdim': [7,5],
             'jitter': 0.02,
             's': 5,
             'font': 16,
             'rotation': 45,
             'xticks': ['MLRM', 'shnMLRM', 'xEEGNet', 'ShallowNet'],
             'ylim': [20,105],
             'linew':2,
             'accrandom': 1/len(classlabels),
             'marker': 'o',
             'linestyle': '--',
             'color': 'tab:blue',
             'loc': 'upper center',
             'figname': 'statModelComparison'}

def stat_comparison(metrics, spec_dict):
    df = pd.DataFrame(metrics)*100
    # Plot setup
    fig, ax = plt.subplots(figsize=(spec_dict['figdim'][0], spec_dict['figdim'][1]))
    
    box = ax.boxplot(df, showfliers=False, widths=0.6, medianprops=dict(color="none"))
    # Add jittered scatter points
    for i, col in enumerate(df.columns, start=1):
        jittered_x = np.random.normal(i, spec_dict['jitter'], size=len(df[col]))
        ax.scatter(jittered_x, df[col], color='black', alpha=1, s=spec_dict['s'])
    
    # Set labels and ticks
    ax.set_ylabel('Median Weighted Accuracy %', fontsize=spec_dict['font']-2, color='tab:blue')
    ax.set_xticks(range(1, len(df.columns) + 1))
    ax.set_xticklabels(spec_dict['xticks'], rotation=spec_dict['rotation'], fontsize=spec_dict['font']-4)
    ax.set_ylim(spec_dict['ylim'][0], spec_dict['ylim'][1])
    ax.axhline(spec_dict['accrandom']*100, linestyle='--', linewidth=spec_dict['linew'], color='red')
    ax.set_title("Comparison with Multinomial Logistic Regression", fontsize=spec_dict['font'])
    
    # Display median values and connect them with a line
    medians = [np.median(df[col]) for col in df.columns]
    for i, median in enumerate(medians, start=1):
        ax.text(i, spec_dict['ylim'][0]+2, f'{median:.1f}%', ha='center', va='bottom',
                color=spec_dict['color'], fontsize=spec_dict['font']-6, fontweight='bold')
    ax.plot(range(1, len(medians) + 1), medians, marker=spec_dict['marker'], 
            linestyle=spec_dict['linestyle'], color=spec_dict['color'], linewidth=spec_dict['linew'])
    
    # Custom legend
    legend_elements = [Line2D([0], [0], marker='.', color='k', label='Single Split', linestyle='None'),
                       Line2D([0], [0], label=f'{spec_dict['accrandom']:.0%} Random Guess', linestyle=spec_dict['linestyle'],
                              linewidth=spec_dict['linew'], color='red')
                      ]
    
    ax.legend(handles=legend_elements, loc=spec_dict['loc'], fontsize=spec_dict['font']-6)
    return fig

fig = stat_comparison(metrics, spec_dict)
# Save and display
if save_img:
    fig.savefig(imgPath + spec_dict['figname'] + imgs_format,
                transparent=False, bbox_inches='tight')

In [ ]:
[ np.mean(i) for i in metrics.values()]